In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates

In [2]:
directory = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "raw")

"/home/emco4286/research/ERCOT_LOAD_DATA/raw"

In [12]:
x = Dict()
i = 1

directory = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "raw")
files = glob("*.csv", directory)
# types_dict = Dict("Hour_End" => DateTime, "HourEnding" => DateTime, "Hour Ending" => DateTime)
date_format_1 = dateformat"mm/dd/yyyy HH:MM"
date_format_2 = dateformat"mm/dd/yyyy HH:MM D\S\T"

for f in files

    df = DataFrame(CSV.File(f)) #, types=types_dict, validate=false))

    if "NCENT" in names(df)
        rename!(df, :NCENT => :NORTH_C)
    end

    if "SCENT" in names(df)
        rename!(df, :SCENT => :SOUTH_C)
    end

    if "SOUTHERN" in names(df)
        rename!(df, :SOUTHERN => :SOUTH)
    end

    if "FWEST" in names(df)
        rename!(df, :FWEST => :FAR_WEST)
    end

    if "HourEnding" in names(df)
        rename!(df, :HourEnding => :Hour_End)
    end

    if "Hour Ending" in names(df)
        rename!(df, "Hour Ending" => :Hour_End)
    end

    # df = df[!, ["Hour_End", "COAST", "NORTH_C"]]

    
    mylist = Vector{DateTime}()
    for dt in df[!, :Hour_End]

        try
            mydt = DateTime(dt, date_format_1)
            push!(mylist, mydt)
        catch e
            mydt = DateTime(dt, date_format_2) - Hour(1)
            push!(mylist, mydt)
            
        end
    end

    df[!, "Hour_End"] = mylist

    x[i] = df
    i +=1
end

merged = reduce(vcat, values(x))
sort!(merged, :Hour_End)

Row,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTH,SOUTH_C,WEST,ERCOT
,DateTime,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,2013-01-01T01:00:00,7607.19,1074.02,1411.82,763.069,10370.3,2206.95,4368.53,883.03,28684.9
2,2013-01-01T02:00:00,7389.27,1035.19,1403.6,754.426,10153.9,2160.08,4233.64,872.532,28002.7
3,2013-01-01T03:00:00,7179.75,1036.22,1395.13,746.216,9989.24,2065.37,4082.9,868.951,27363.8
4,2013-01-01T04:00:00,7039.69,1032.78,1395.59,749.029,9947.86,1991.15,4010.52,865.799,27032.4
5,2013-01-01T05:00:00,6991.41,1042.91,1401.24,757.148,10097.4,1954.96,4038.68,879.988,27163.8
6,2013-01-01T06:00:00,7050.73,1069.29,1419.56,777.767,10472.1,1962.08,4174.05,911.222,27836.8
7,2013-01-01T07:00:00,7177.79,1109.14,1444.43,793.098,10891.2,1994.51,4360.99,950.809,28722.0
8,2013-01-01T08:00:00,7223.21,1148.8,1463.21,812.598,11190.2,2041.97,4522.13,989.594,29391.7
9,2013-01-01T09:00:00,7314.25,1196.98,1474.82,839.631,11627.1,2131.51,4688.33,1021.13,30293.8


In [14]:
names(merged)[2:end]

9-element Vector{String}:
 "COAST"
 "EAST"
 "FAR_WEST"
 "NORTH"
 "NORTH_C"
 "SOUTH"
 "SOUTH_C"
 "WEST"
 "ERCOT"

In [17]:
my_names = names(merged)[2:end]

for n in my_names

    s = merged[!, n]
    mylist = Vector{Float64}()
    i = 0

    for v in s
        if v isa String15
            v2 = replace(v, ','=>"", " " =>"")
            v3 = parse.(Float64, v2)
        elseif v isa Missing
            v3 = NaN64
        elseif typeof(v) != Float64
            v3 = parse.(Float64, v)
        else
            v3 = v
        end
        push!(mylist, v3)
        i += 1
    end

    merged[!, n] = mylist
end

In [18]:
merged

Row,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTH,SOUTH_C,WEST,ERCOT
,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2013-01-01T01:00:00,7607.19,1074.02,1411.82,763.069,10370.3,2206.95,4368.53,883.03,28684.9
2,2013-01-01T02:00:00,7389.27,1035.19,1403.6,754.426,10153.9,2160.08,4233.64,872.532,28002.7
3,2013-01-01T03:00:00,7179.75,1036.22,1395.13,746.216,9989.24,2065.37,4082.9,868.951,27363.8
4,2013-01-01T04:00:00,7039.69,1032.78,1395.59,749.029,9947.86,1991.15,4010.52,865.799,27032.4
5,2013-01-01T05:00:00,6991.41,1042.91,1401.24,757.148,10097.4,1954.96,4038.68,879.988,27163.8
6,2013-01-01T06:00:00,7050.73,1069.29,1419.56,777.767,10472.1,1962.08,4174.05,911.222,27836.8
7,2013-01-01T07:00:00,7177.79,1109.14,1444.43,793.098,10891.2,1994.51,4360.99,950.809,28722.0
8,2013-01-01T08:00:00,7223.21,1148.8,1463.21,812.598,11190.2,2041.97,4522.13,989.594,29391.7
9,2013-01-01T09:00:00,7314.25,1196.98,1474.82,839.631,11627.1,2131.51,4688.33,1021.13,30293.8


In [19]:
fname = "ercot_hourly_load_2013_2023.csv"
path = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "processed", fname)
CSV.write(path, merged)

"/home/emco4286/research/ERCOT_LOAD_DATA/processed/ercot_hourly_load_2013_2023.csv"